# Python Pydantic

Pydantic ist eine erweiterung der Normalen Dataclasses.
Es ermöglicht Datenvalidierung und Einstellungsmanagement mit Python-Typ-Annotationen.
pydantic erzwingt Typisierung zur Laufzeit und liefert benutzerfreundliche Fehlermeldungen, wenn Daten ungültig sind.



In [2]:
import json


def main() -> None:
    with open('../../Materialien/books.json') as book_file:
        data = json.load(book_file)

        print(data[0])


if __name__ == '__main__':
    main()


{'title': 'Zero to One', 'subtitle': 'Notes on Startups, or How to Build the Future', 'author': 'Peter Thiel', 'publisher': 'Ballantine Books', 'isbn_10': '0753555190', 'isbn_13': '978-0753555194', 'price': 14.29, 'author2': {'name': 'Peter Thiel', 'verified': True}}


In [4]:
from pydantic import BaseModel

class Book(BaseModel):
    title: str
    author: str
    publisher: str
    price: float

def main() -> None:
    with open('../../Materialien/books.json') as book_file:
        data = json.load(book_file)
        books: list[Book] = [Book(**book) for book in data]
        print(books[0])
        print(books[0].author)


if __name__ == '__main__':
    main()


title='Zero to One' author='Peter Thiel' publisher='Ballantine Books' price=14.29
Peter Thiel


In [43]:
from pydantic import BaseModel, validator
from typing import Optional

class ISBN10FormatError(Exception):
    def __init__(self, isbn: str, message:str):
        self.isbn = isbn
        self.message = message
        super().__init__(message)

class Book(BaseModel):
    title: str
    author: str
    publisher: str
    price: float
    isbn_10: Optional[str]
    isbn_13: Optional[str]
    subtitle: Optional[str]
    
    @validator('isbn_10')
    @classmethod
    def validate_isbn10(cls, isbn: str):
        chars = [char for char in isbn if char in "0123456789xX"]
        if len(chars) != 10:
            raise ISBN10FormatError(isbn, "ISBN10 muss 10 zahlen haben.")

        def char_to_int(char: str) -> int:
            if char in "xX":
                return 10
            return int(char)
        
        isbn_sum = sum((10 - index) * char_to_int(char) for index, char in enumerate(chars))
        
        if isbn_sum % 11 != 0:
            raise ISBN10FormatError(isbn, "Die Summe der ISBN10 zahlen muss durch 11 teilbar sein.")

def main() -> None:
    with open('../../Materialien/books.json') as book_file:
        data = json.load(book_file)
        books: list[Book] = [Book(**book) for book in data]
        print(books[0])
        print(books[0].author)


if __name__ == '__main__':
    main()


title='Zero to One' author='Peter Thiel' publisher='Ballantine Books' price=14.29 isbn_10=None isbn_13=None subtitle='Notes on Startups, or How to Build the Future'
Peter Thiel


In [45]:
import pydantic
from typing import Optional


class ISBN10FormatError(Exception):
    def __init__(self, isbn: str, message: str) -> None:
        self.message = message
        self.isbn = isbn
        super().__init__(message)


class ISBNMissingError(Exception):
    def __init__(self, title: str, message: str) -> None:
        self.message = message
        self.title = title
        super().__init__(message)


class Book(pydantic.BaseModel):
    title: str
    author: str
    publisher: str
    price: float
    isbn_10: Optional[str]
    isbn_13: Optional[str]
    subtitle: Optional[str]

    @pydantic.root_validator(pre=True)
    @classmethod
    def check_isbn10_or_isbn13(cls, values: dict[str, str]):
        if "isbn_10" not in values and "isbn_13" not in values:
            raise ISBNMissingError(
                title=values['title'], message="Das Buch sollte entweder eine ISBN10 oder ISBN13 haben.")
        return values

    @pydantic.validator("isbn_10")
    @classmethod
    def isbn_10_valid(cls, isbn: str):
        chars = [char for char in isbn if char in "0123456789xX"]
        if len(chars) != 10:
            raise ISBN10FormatError(
                isbn=isbn, message="ISBN10 muss 10 zahlen haben.")

        def char_to_int(char: str) -> int:
            if char in "xX":
                return 10
            return int(char)

        isbn_sum = sum((10 - index) * char_to_int(char)
                       for index, char in enumerate(chars))
        if isbn_sum % 11 != 0:
            raise ISBN10FormatError(
                isbn=isbn, message="Die Summe der ISBN10 zahlen muss durch 11 teilbar sein.")


def main() -> None:
    with open('../../Materialien/books.json') as book_file:
        data = json.load(book_file)
        books: list[Book] = [Book(**item) for item in data]
        print(books[0].author)


if __name__ == '__main__':
    main()


peter thiel


In [47]:
import pydantic
from typing import Optional


class ISBN10FormatError(Exception):
    def __init__(self, isbn: str, message: str) -> None:
        self.message = message
        self.isbn = isbn
        super().__init__(message)


class ISBNMissingError(Exception):
    def __init__(self, title: str, message: str) -> None:
        self.message = message
        self.title = title
        super().__init__(message)


class Book(pydantic.BaseModel):
    title: str
    author: str
    publisher: str
    price: float
    isbn_10: Optional[str]
    isbn_13: Optional[str]
    subtitle: Optional[str]

    @pydantic.root_validator(pre=True)
    @classmethod
    def check_isbn10_or_isbn13(cls, values: dict[str, str]):
        if "isbn_10" not in values and "isbn_13" not in values:
            raise ISBNMissingError(
                title=values['title'], message="Das Buch sollte entweder eine ISBN10 oder ISBN13 haben.")
        return values

    @pydantic.validator("isbn_10")
    @classmethod
    def isbn_10_valid(cls, isbn: str):
        chars = [char for char in isbn if char in "0123456789xX"]
        if len(chars) != 10:
            raise ISBN10FormatError(
                isbn=isbn, message="ISBN10 muss 10 zahlen haben.")

        def char_to_int(char: str) -> int:
            if char in "xX":
                return 10
            return int(char)

        isbn_sum = sum((10 - index) * char_to_int(char)
                       for index, char in enumerate(chars))
        if isbn_sum % 11 != 0:
            raise ISBN10FormatError(
                isbn=isbn, message="Die Summe der ISBN10 zahlen muss durch 11 teilbar sein.")

    class Config:
        allow_mutation = False  # make book imutable
        anystr_lower = True  # make all strings lowercase

def main() -> None:
    with open('../../Materialien/books.json') as book_file:
        data = json.load(book_file)
        books: list[Book] = [Book(**item) for item in data]
        print(books[0])
        books[0].price = 10.0


if __name__ == '__main__':
    main()


title='zero to one' author='peter thiel' publisher='ballantine books' price=14.29 isbn_10=None isbn_13='978-0753555194' subtitle='notes on startups, or how to build the future'


TypeError: "Book" is immutable and does not support item assignment